In [20]:
import sys
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+

from pyspark.sql import SparkSession, functions, types
from pyspark.sql.types import StructField, StringType, StructType
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import col, count, first
from pyspark.ml.feature import Imputer
from pyspark.sql.types import IntegerType, BooleanType, DateType
from pyspark.sql.functions import when, col, avg

In [21]:
spark = SparkSession.builder \
    .appName("CrashRemedyPrediction") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "6g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.sql.debug.maxToStringFields", "200") \
    .getOrCreate()

In [22]:
df = spark.read.parquet('/Users/swaifahaque/Documents/BigDataProject/merged')

In [24]:
df.show(5)

+------------+----+-------+--------------+-------------------+--------------+-------------------+--------------+------+---------------------+------+-----------+-------------+---------+-----------------+-----------+-----------------+-------------------+--------------------+--------------+-------+------------+----------------+--------------+--------------------+---------------+-----------------------+----------------+--------------------+----------------+-------------+---------------+---------------+-----------+-----------------+--------------+------------------+--------------------+--------------------+--------------------+-----------+----------+-------------+-----------+------+
|municipality|year|  month|        region|crash_configuration|total_casualty|pedestrian_involved|crash_severity|   day|is_intersection_crash|street|       time|total_crashes| latitude|     cross_street|  longitude|             city|     collision_type|               light|road_condition|weather|road_surface|spee

In [26]:
# Step 1: Drop missing or invalid lat/lon rows if needed
df = df.dropna(subset=['latitude', 'longitude'])


In [28]:
from pyspark.sql import functions as F

# Optional: round lat/lon to cluster nearby crashes (e.g., to 4 decimal places)
df = df.withColumn("lat_rounded", F.round(F.col("latitude"), 4))
df = df.withColumn("lon_rounded", F.round(F.col("longitude"), 4))

In [29]:
# Group by location and count
location_counts = df.groupBy("lat_rounded", "lon_rounded") \
                    .agg(F.count("*").alias("total_crashes"))

In [30]:
# Calculate thresholds for classification using approxQuantile
low_thresh, high_thresh = location_counts.approxQuantile("total_crashes", [0.33, 0.66], 0)

In [31]:
# Define UDF to classify hotspot levels
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def classify_hotspot(count):
    if count <= low_thresh:
        return "Low"
    elif count <= high_thresh:
        return "Moderate"
    else:
        return "High"


In [32]:
classify_udf = udf(classify_hotspot, StringType())

In [33]:
# Add hotspot level
location_counts = location_counts.withColumn("hotspot_level", classify_udf(F.col("total_crashes")))


In [34]:
# Optional: join back to original DataFrame
df_with_hotspot = df.join(location_counts, on=["lat_rounded", "lon_rounded"], how="left")

In [36]:
# Rename total_crashes in location_counts to avoid conflict
location_counts = location_counts.withColumnRenamed("total_crashes", "location_total_crashes")

# Join back to original dataframe
df_with_hotspot = df.join(location_counts, on=["lat_rounded", "lon_rounded"], how="left")

# Now you can safely select
df_with_hotspot.select(
    "latitude", "longitude", "location_total_crashes", "hotspot_level"
).show()

+---------+-----------+----------------------+-------------+
| latitude|  longitude|location_total_crashes|hotspot_level|
+---------+-----------+----------------------+-------------+
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381|                 61599|         High|
|49.033431|-122.266381| 

Top Hostspots in BC

In [42]:
from pyspark.sql import functions as F

# Group by location (with optional city/municipality) and count crashes
top_hotspots = df_with_hotspot.groupBy(
    "lat_rounded", "lon_rounded", "municipality"
).agg(
    F.count("*").alias("total_crashes")
).orderBy(F.desc("total_crashes"))

# Show top 10 locations with most crashes
top_hotspots.show(10, truncate=False)

+-----------+-----------+------------+-------------+
|lat_rounded|lon_rounded|municipality|total_crashes|
+-----------+-----------+------------+-------------+
|49.2576    |-123.0041  |burnaby     |614123       |
|49.1627    |-122.8458  |surrey      |567345       |
|49.1921    |-122.7778  |surrey      |535989       |
|49.1016    |-122.4928  |langley     |529655       |
|49.1626    |-122.868   |surrey      |436282       |
|49.119     |-122.6914  |surrey      |430529       |
|49.2461    |-122.9142  |burnaby     |426562       |
|49.2582    |-123.0236  |burnaby     |425388       |
|49.1764    |-122.7347  |surrey      |402742       |
|49.1336    |-122.8456  |surrey      |402011       |
+-----------+-----------+------------+-------------+
only showing top 10 rows



Visualizing the hotspots in BC using a HeatMap

In [38]:
unique_locations = df_with_hotspot.select(
    "lat_rounded", "lon_rounded", "location_total_crashes"
).dropna().distinct()

# Now it's much smaller and safe to collect
pandas_df = unique_locations.toPandas()

In [41]:
import folium
from folium.plugins import HeatMap

# Step 1: Create the map centered around average location
m = folium.Map(
    location=[pandas_df["lat_rounded"].mean(), pandas_df["lon_rounded"].mean()],
    zoom_start=10,
    tiles="CartoDB dark_matter"  # You can change this to "OpenStreetMap" if preferred
)

# Step 2: Build the heatmap data list
# Use location_total_crashes as weight directly
heat_data = [
    [row["lat_rounded"], row["lon_rounded"], row["location_total_crashes"]]
    for _, row in pandas_df.iterrows()
]

# Step 3: Add heatmap layer
HeatMap(heat_data, radius=10, blur=15, max_zoom=1).add_to(m)

# Step 4: Save to HTML
m.save("collision_hotspots_heatmap.html")

# Optional: View in notebook (if in Jupyter)
m